<a href="https://colab.research.google.com/github/NiranjanPrabakar/Machine-Learning-for-Robotics/blob/main/Model_Evaluation_Metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import xgboost as xgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# 1. Load the data
def load_and_preprocess_data(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path)

    # Filter the data for TAMIL NADU only
    tamil_nadu_data = df[df['SUBDIVISION'] == 'TAMIL NADU']

    # Drop rows with null values
    tamil_nadu_data = tamil_nadu_data.dropna()

    # Reset index
    tamil_nadu_data = tamil_nadu_data.reset_index(drop=True)

    # Create a new dataframe with YEAR and monthly data
    months = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']

    # Calculate the average monthly rainfall
    monthly_rainfall = tamil_nadu_data[['YEAR'] + months]

    # Create a time series dataframe
    time_series_data = monthly_rainfall.copy()

    # Add annual rainfall as target
    time_series_data['ANNUAL'] = tamil_nadu_data['ANNUAL']

    return time_series_datadata

# 2. Prepare data for time series forecasting
def prepare_time_series_data(data, n_steps=5):
    # Create sequences for LSTM
    X, y = [], []
    for i in range(len(data) - n_steps):
        # Create sequence of n_steps years' monthly data
        sequence = data.iloc[i:i+n_steps, 1:13].values.flatten()
        X.append(sequence)
        # Use the annual rainfall of the next year as target
        y.append(data.iloc[i+n_steps, -1])

    return np.array(X), np.array(y)

# 3. Create and train LSTM model
def create_lstm_model(X_train):
    # Reshape input for LSTM [samples, timesteps, features]
    n_samples = X_train.shape[0]
    n_steps = 5
    n_features = 12  # 12 months
    X_train_reshaped = X_train.reshape(n_samples, n_steps, n_features)

    # Create LSTM model
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features), return_sequences=True))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    return model, X_train_reshaped

# 4. Create and train hybrid models and baseline models
def train_models(X_train, y_train, X_val, y_val, X_test, y_test):
    # LSTM model
    lstm_model, X_train_reshaped = create_lstm_model(X_train)
    X_val_reshaped = X_val.reshape(X_val.shape[0], 5, 12)

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train LSTM model
    history = lstm_model.fit(
        X_train_reshaped, y_train,
        epochs=100,
        batch_size=32,
        validation_data=(X_val_reshaped, y_val),
        callbacks=[early_stopping],
        verbose=0
    )

    # Get LSTM predictions on training and validation data for hybrid models
    X_train_reshaped = X_train.reshape(X_train.shape[0], 5, 12)
    X_val_reshaped = X_val.reshape(X_val.shape[0], 5, 12)
    X_test_reshaped = X_test.reshape(X_test.shape[0], 5, 12)

    lstm_train_preds = lstm_model.predict(X_train_reshaped).flatten()
    lstm_val_preds = lstm_model.predict(X_val_reshaped).flatten()
    lstm_test_preds = lstm_model.predict(X_test_reshaped).flatten()

    # Create features for hybrid models by combining original features with LSTM predictions
    X_train_hybrid = np.column_stack((X_train, lstm_train_preds))
    X_val_hybrid = np.column_stack((X_val, lstm_val_preds))
    X_test_hybrid = np.column_stack((X_test, lstm_test_preds))

    # Initialize models
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
    xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=42)

    # Hybrid model: LSTM + RF
    lstm_rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    lstm_rf_model.fit(X_train_hybrid, y_train)

    # Hybrid model: LSTM + XGBoost
    lstm_xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=42)
    lstm_xgb_model.fit(X_train_hybrid, y_train)

    # Train standalone models
    rf_model.fit(X_train, y_train)
    gb_model.fit(X_train, y_train)
    xgb_model.fit(X_train, y_train)

    return {
        'lstm': lstm_model,
        'rf': rf_model,
        'gb': gb_model,
        'xgb': xgb_model,
        'lstm_rf': lstm_rf_model,
        'lstm_xgb': lstm_xgb_model
    }, X_test_reshaped, X_test_hybrid

# 5. Evaluate models
def evaluate_models(models, X_test, X_test_reshaped, X_test_hybrid, y_test):
    results = {}

    # Evaluate LSTM
    lstm_preds = models['lstm'].predict(X_test_reshaped).flatten()
    results['LSTM'] = {
        'predictions': lstm_preds,
        'mae': mean_absolute_error(y_test, lstm_preds),
        'mse': mean_squared_error(y_test, lstm_preds),
        'rmse': np.sqrt(mean_squared_error(y_test, lstm_preds)),
        'r2': r2_score(y_test, lstm_preds)
    }

    # Evaluate Random Forest
    rf_preds = models['rf'].predict(X_test)
    results['Random Forest'] = {
        'predictions': rf_preds,
        'mae': mean_absolute_error(y_test, rf_preds),
        'mse': mean_squared_error(y_test, rf_preds),
        'rmse': np.sqrt(mean_squared_error(y_test, rf_preds)),
        'r2': r2_score(y_test, rf_preds)
    }

    # Evaluate Gradient Boosting
    gb_preds = models['gb'].predict(X_test)
    results['Gradient Boosting'] = {
        'predictions': gb_preds,
        'mae': mean_absolute_error(y_test, gb_preds),
        'mse': mean_squared_error(y_test, gb_preds),
        'rmse': np.sqrt(mean_squared_error(y_test, gb_preds)),
        'r2': r2_score(y_test, gb_preds)
    }

    # Evaluate XGBoost
    xgb_preds = models['xgb'].predict(X_test)
    results['XGBoost'] = {
        'predictions': xgb_preds,
        'mae': mean_absolute_error(y_test, xgb_preds),
        'mse': mean_squared_error(y_test, xgb_preds),
        'rmse': np.sqrt(mean_squared_error(y_test, xgb_preds)),
        'r2': r2_score(y_test, xgb_preds)
    }

    # Evaluate LSTM + RF
    lstm_rf_preds = models['lstm_rf'].predict(X_test_hybrid)
    results['LSTM + RF'] = {
        'predictions': lstm_rf_preds,
        'mae': mean_absolute_error(y_test, lstm_rf_preds),
        'mse': mean_squared_error(y_test, lstm_rf_preds),
        'rmse': np.sqrt(mean_squared_error(y_test, lstm_rf_preds)),
        'r2': r2_score(y_test, lstm_rf_preds)
    }

    # Evaluate LSTM + XGBoost
    lstm_xgb_preds = models['lstm_xgb'].predict(X_test_hybrid)
    results['LSTM + XGBoost'] = {
        'predictions': lstm_xgb_preds,
        'mae': mean_absolute_error(y_test, lstm_xgb_preds),
        'mse': mean_squared_error(y_test, lstm_xgb_preds),
        'rmse': np.sqrt(mean_squared_error(y_test, lstm_xgb_preds)),
        'r2': r2_score(y_test, lstm_xgb_preds)
    }

    return results

# 6. Forecast for future years (2016-2030)
def forecast_future(data, models, n_steps=5, future_years=15):
    # Extract the last n_steps years of data for prediction
    last_years = data.iloc[-n_steps:].copy()

    # Create predictions for future years
    future_predictions = {}
    for model_name in ['Random Forest', 'Gradient Boosting', 'LSTM + RF', 'LSTM + XGBoost']:
        future_predictions[model_name] = []

    # Current last year
    last_year = data['YEAR'].iloc[-1]

    for i in range(future_years):
        future_year = last_year + i + 1

        # Use the last n_steps years to predict
        last_sequence = last_years.iloc[:, 1:13].values.flatten()

        # Make predictions with each model
        # Random Forest
        rf_pred = models['rf'].predict([last_sequence])[0]
        future_predictions['Random Forest'].append(rf_pred)

        # Gradient Boosting
        gb_pred = models['gb'].predict([last_sequence])[0]
        future_predictions['Gradient Boosting'].append(gb_pred)

        # LSTM prediction
        lstm_sequence = last_sequence.reshape(1, n_steps, 12)
        lstm_pred = models['lstm'].predict(lstm_sequence)[0][0]

        # Combine LSTM prediction with original features for hybrid models
        hybrid_features = np.append(last_sequence, lstm_pred)

        # LSTM + RF
        lstm_rf_pred = models['lstm_rf'].predict([hybrid_features])[0]
        future_predictions['LSTM + RF'].append(lstm_rf_pred)

        # LSTM + XGBoost
        lstm_xgb_pred = models['lstm_xgb'].predict([hybrid_features])[0]
        future_predictions['LSTM + XGBoost'].append(lstm_xgb_pred)

        # Update the last years data by removing the oldest year and adding a placeholder for the newest
        # This is an approximation - in real scenarios this would be more complex
        new_year_data = last_years.iloc[1:].copy()
        new_row = pd.DataFrame({
            'YEAR': [future_year],
            'JAN': [last_years['JAN'].mean()],
            'FEB': [last_years['FEB'].mean()],
            'MAR': [last_years['MAR'].mean()],
            'APR': [last_years['APR'].mean()],
            'MAY': [last_years['MAY'].mean()],
            'JUN': [last_years['JUN'].mean()],
            'JUL': [last_years['JUL'].mean()],
            'AUG': [last_years['AUG'].mean()],
            'SEP': [last_years['SEP'].mean()],
            'OCT': [last_years['OCT'].mean()],
            'NOV': [last_years['NOV'].mean()],
            'DEC': [last_years['DEC'].mean()],
            'ANNUAL': [rf_pred]  # Using the RF prediction as an approximation
        })
        last_years = pd.concat([new_year_data, new_row], ignore_index=True)

    # Create a DataFrame with future predictions
    future_df = pd.DataFrame({
        'YEAR': range(last_year + 1, last_year + future_years + 1),
        'Random Forest': future_predictions['Random Forest'],
        'Gradient Boosting': future_predictions['Gradient Boosting'],
        'LSTM + RF': future_predictions['LSTM + RF'],
        'LSTM + XGBoost': future_predictions['LSTM + XGBoost']
    })

    return future_df

# 7. Visualize results
def visualize_results(data, results, future_df, test_years, y_test):
    # FIX: Get actual test years from the predictions length
    actual_test_years = test_years[-len(results['Random Forest']['predictions']):]

    # 1. Plot model performance metrics
    metrics = ['mae', 'mse', 'rmse', 'r2']
    fig, axes = plt.subplots(2, 2, figsize=(18, 12))
    axes = axes.flatten()

    for i, metric in enumerate(metrics):
        metric_values = [results[model][metric] for model in ['Random Forest', 'Gradient Boosting', 'LSTM + RF', 'LSTM + XGBoost']]
        ax = axes[i]
        bars = ax.bar(['Random Forest', 'Gradient Boosting', 'LSTM + RF', 'LSTM + XGBoost'], metric_values)
        ax.set_title(f'{metric.upper()} Comparison', fontsize=16)
        ax.set_ylabel(metric.upper())
        ax.tick_params(axis='x', rotation=45)

        # Add values on top of bars
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                    f'{height:.4f}', ha='center', va='bottom', fontsize=10)

    plt.tight_layout()
    plt.savefig('model_performance_metrics.png', dpi=300)
    plt.close()

    # 2. Plot actual vs predicted rainfall for each model (2001-2015)
    fig, axes = plt.subplots(2, 2, figsize=(18, 12))
    axes = axes.flatten()

    models_to_plot = ['Random Forest', 'Gradient Boosting', 'LSTM + RF', 'LSTM + XGBoost']

    # Get the actual test values that match prediction length
    y_test_adjusted = y_test[-len(results['Random Forest']['predictions']):]

    for i, model_name in enumerate(models_to_plot):
        ax = axes[i]
        ax.plot(actual_test_years, results[model_name]['predictions'], marker='o', label=f'{model_name} Predicted')
        ax.plot(actual_test_years, y_test_adjusted, marker='x', linestyle='--', label='Actual')
        ax.set_title(f'{model_name}: Actual vs Predicted Rainfall (2001-2015)', fontsize=16)
        ax.set_xlabel('Year')
        ax.set_ylabel('Annual Rainfall (mm)')
        ax.legend()
        ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('actual_vs_predicted_2001_2015.png', dpi=300)
    plt.close()

    # 3. Plot future predictions (2016-2030)
    plt.figure(figsize=(15, 8))
    for model_name in models_to_plot:
        plt.plot(future_df['YEAR'], future_df[model_name], marker='o', label=model_name)

    plt.title('Rainfall Predictions (2016-2030)', fontsize=18)
    plt.xlabel('Year', fontsize=14)
    plt.ylabel('Annual Rainfall (mm)', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.savefig('future_predictions_2016_2030.png', dpi=300)
    plt.close()

    # 4. Combined historical and future predictions plot
    # Extract the actual values for years 2001-2015
    historical_years = actual_test_years
    historical_actual = y_test_adjusted

    # Create the future years
    future_years = future_df['YEAR'].values

    plt.figure(figsize=(18, 10))

    # Plot historical actual values
    plt.plot(historical_years, historical_actual, 'k-', marker='o', linewidth=2, label='Actual (2001-2015)')

    # Plot historical predictions for each model
    for model_name in models_to_plot:
        historical_pred = results[model_name]['predictions']
        plt.plot(historical_years, historical_pred, linestyle='--', marker='x', alpha=0.7, label=f'{model_name} (2001-2015)')

    # Add a vertical line to separate historical and future
    plt.axvline(x=historical_years[-1], color='r', linestyle='--', alpha=0.5)
    plt.text(historical_years[-1] + 0.5, plt.ylim()[0] + 0.9*(plt.ylim()[1] - plt.ylim()[0]), 'Predictions Start',
             rotation=90, color='r')

    # Plot future predictions for each model
    for model_name in models_to_plot:
        future_pred = future_df[model_name].values
        plt.plot(future_years, future_pred, linestyle='-', marker='o', label=f'{model_name} (2016-2030)')

    plt.title('Historical vs Future Rainfall Predictions for Tamil Nadu', fontsize=20)
    plt.xlabel('Year', fontsize=16)
    plt.ylabel('Annual Rainfall (mm)', fontsize=16)
    plt.grid(True, alpha=0.3)
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, fontsize=12)
    plt.tight_layout()
    plt.savefig('historical_and_future_predictions.png', dpi=300)
    plt.close()

    return

# Main execution
if __name__ == '__main__':
    # Define file path
    file_path = "/content/drive/MyDrive/rainfall in india 1901-2015.csv"

    # Load and preprocess data
    print("Loading and preprocessing data...")
    data = load_and_preprocess_data(file_path)

    # Split data into training (1901-2000) and testing (2001-2015)
    train_data = data[data['YEAR'] <= 2000]
    test_data = data[data['YEAR'] > 2000]

    # Get the test years for plotting
    test_years = test_data['YEAR'].values

    # Prepare time series data
    n_steps = 5

    # Prepare training data
    X_train_val, y_train_val = prepare_time_series_data(train_data, n_steps)
    X_test, y_test = prepare_time_series_data(test_data, n_steps)

    # Further split training data into training and validation
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

    # Train models
    print("Training models...")
    models, X_test_reshaped, X_test_hybrid = train_models(X_train, y_train, X_val, y_val, X_test, y_test)

    # Evaluate models
    print("Evaluating models...")
    results = evaluate_models(models, X_test, X_test_reshaped, X_test_hybrid, y_test)

    # Print evaluation metrics
    print("\nModel Evaluation Metrics:")
    for model_name, metrics in results.items():
        print(f"\n{model_name}:")
        print(f"  MAE: {metrics['mae']:.4f}")
        print(f"  MSE: {metrics['mse']:.4f}")
        print(f"  RMSE: {metrics['rmse']:.4f}")
        print(f"  R²: {metrics['r2']:.4f}")

    # Find the best model based on RMSE
    best_model = min(results.items(), key=lambda x: x[1]['rmse'])[0]
    print(f"\nBest model based on RMSE: {best_model}")

    # Forecast future rainfall (2016-2030)
    print("\nForecasting future rainfall (2016-2030)...")
    future_df = forecast_future(data, models, n_steps=n_steps, future_years=15)

    # Print future predictions
    print("\nFuture Rainfall Predictions (2016-2030):")
    print(future_df)

    # Visualize results
    print("\nVisualizing results...")
    # Pass y_test as an additional parameter to ensure correct shapes
    visualize_results(data, results, future_df, test_years, y_test)

    print("\nAnalysis complete! Check the generated visualizations for insights.")

Loading and preprocessing data...
Training models...
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 403ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Evaluating models...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step

Model Evaluation Metrics:

LSTM:
  MAE: 243.7302
  MSE: 104454.1723
  RMSE: 323.1937
  R²: -2.4082

Random Forest:
  MAE: 141.6161
  MSE: 34619.6983
  RMSE: 186.0637
  R²: -0.1296

Gradient Boosting:
  MAE: 152.5883
  MSE: 38715.7670
  RMSE: 196.7632
  R²: -0.2632

XGBoost:
  MAE: 181.5959
  MSE: 53556.9000
  RMSE: 231.4236
  R²: -0.7475

LSTM + RF:
  MAE: 148.1449
  MSE: 37025.0415
  RMSE: 192.4189
  R²: -0.2081

LSTM + XGBoost:
  MAE: 180.2928
  MSE: 53444.1157
  RMSE: 231.1798
  R²: -0.7438

Best model based on RMSE: Random Forest

Forecasting future rainfall (2016-2030)...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/ste